Dataset used:

[Munich Germany](https://mapzen.com/data/metro-extracts/metro/munich_germany/) from OpenStreetMap.  
[Download(OSMXML 39MB)](https://s3.amazonaws.com/metro-extracts.mapzen.com/munich_germany.osm.bz2)